In [1]:
# !pip install -q mediapipe

In [2]:
import cv2
import numpy as np
import time
# from google.colab.patches import cv2_imshow
import PoseModule as pm

In [3]:
def resize_video(input_path, output_path, times):
    cap = cv2.VideoCapture(input_path)
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(original_width / times)
    height = int(original_height / times)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (width, height))
        out.write(resized_frame)

    cap.release()
    out.release()

    print("Video resizing complete.")

# If input video is with high resolution we need resize

# input_path = "20231220_113403_001.mp4"
# output_path = "output_video.mp4"
# resize_video(input_path, output_path, times = 3)

In [93]:
def aitrainer(feedback_type: str, body: str, level: str):
    if feedback_type == 'real_time':
        cap = cv2.VideoCapture(0)
        save_video = False
    else:
        cap = cv2.VideoCapture(feedback_type)
        save_video = True
    detector = pm.poseDetector()
    count = 0
    dir_ = 0
    pTime = 0

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(f'aitrainer_mediapipe_{feedback_type}.mp4', fourcc, 30.0, (width, height))

    while True:
        success, img = cap.read()
        if not success:
            break
        if feedback_type == 'real_time':
            img = cv2.flip(img, 1)
        img = detector.findPose(img, False)
        lmList = detector.findPosition(img, False)

        if len(lmList) != 0:
            if body == 'right_arm':
                angle = detector.findAngle(img, 11, 13, 15)
            if body == 'left_arm':
                angle = detector.findAngle(img, 12, 14, 16)
            if body == 'right_foot':
                angle = detector.findAngle(img, 23, 25, 27)
            if body == 'left_foot':
                angle = detector.findAngle(img, 24, 26, 28)
            if body == 'abs':
                angle = detector.findAngle(img, 25, 23, 11)
            
            if level == 'easy':
                per = np.interp(angle, (80, 110), (0, 100))
                bar = np.interp(angle, (80, 110), (300, 100))
            if level == 'medium':
                per = np.interp(angle, (60, 130), (0, 100))
                bar = np.interp(angle, (60, 130), (300, 100))
            if level == 'hard':
                per = np.interp(angle, (45, 150), (0, 100))
                bar = np.interp(angle, (45, 150), (300, 100))
            
            color = (0, 255, 255)
            if per == 100:
                color = (0, 255, 0)
                if dir_ == 0:
                    count += 0.5
                    dir_ = 1
            if per == 0:
                color = (0, 0, 255)
                if dir_ == 1:
                    count += 0.5
                    dir_ = 0

            cv2.rectangle(img, (10, 100), (50, 300), color, 3)
            cv2.rectangle(img, (10, int(bar)), (50, 300), color, cv2.FILLED)
            cv2.putText(img, f'{int(per)}%', (10, 75), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)

            text = f'count: {int(count)}'
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 3, 3)
            cv2.rectangle(img, (80, 80 - text_size[1]), (text_size[0] + 90, 80), (255, 255, 255), cv2.FILLED)
            cv2.putText(img, text, (80, 80), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        fps_text = f'fps: {int(fps)}'
        fps_text_size, _ = cv2.getTextSize(fps_text, cv2.FONT_HERSHEY_PLAIN, 3, 3)
        cv2.rectangle(img, (80, 40 - fps_text_size[1]), (fps_text_size[0] + 90, 40), (255, 255, 255), cv2.FILLED)
        cv2.putText(img, fps_text, (80, 40), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
        out.write(img)

        if save_video:
            out.write(img)

        cv2.imshow("YourAiTrainer", img)
        cv2.waitKey(1)

    cap.release()
    if save_video:
        out.release()
    cv2.destroyAllWindows()

In [94]:
aitrainer('output_video.mp4', 'right_foot', 'easy')

In [70]:
def calculate_angle(A, B, C):
    BA = A - B
    BC = C - B
    cosine_angle = np.dot(BA, BC) / (np.linalg.norm(BA) * np.linalg.norm(BC))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

def find_most_active_points(points, last_points):
    if points.shape[0] != last_points.shape[0]:
        return points
    else:
        displacements = np.linalg.norm(points - last_points, axis=1)
        active_indices = np.argsort(displacements)[-3:]

        return points[active_indices]

def detect_keypoints_dummy(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create()
    keypoints = orb.detect(gray, None)
    keypoints = np.array([kp.pt for kp in keypoints])
    return keypoints

def main():
    cap = cv2.VideoCapture('video_2023-12-20_13-20-51.mp4')
    ret, last_frame = cap.read()
    last_points = detect_keypoints_dummy(last_frame)
    fps_ = cap.get(cv2.CAP_PROP_FPS)
    delay = int(1000 / fps_)
    count = 0
    pTime = 0

    # Obtain frame width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec (codec may vary based on system configuration)
    out = cv2.VideoWriter('output_comparison_1.mp4', fourcc, 30.0, (frame_width, frame_height))  # Create VideoWriter object
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        current_points = detect_keypoints_dummy(frame)
        active_points = find_most_active_points(current_points, last_points)
        last_points = current_points
        if len(active_points) == 3:
            active_points = np.round(active_points).astype(int)
            angle = calculate_angle(active_points[0], active_points[1], active_points[2])
            if abs(180 - angle) > 179.65:
                count += 0.5
            elif abs(180 - angle) < 0.35:
                count += 0.5
            cv2.putText(frame, f"Count: {int(count)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        
        cv2.putText(frame, f"FPS: {round(fps, 4)}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
        
        # Write the frame to the video file
        out.write(frame)

        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break

    # Release the VideoWriter object and capture object
    out.release()
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

C:\Users\Mekan\AppData\Local\Temp\ipykernel_9004\238239944.py:8: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))
C:\Users\Mekan\AppData\Local\Temp\ipykernel_9004\238239944.py:7: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(BA, BC) / (np.linalg.norm(BA) * np.linalg.norm(BC))


In [71]:
# Concatenate and save videoes

cap1 = cv2.VideoCapture('aitrainer_comparison_1.mp4')
cap2 = cv2.VideoCapture('output_comparison_1.mp4')

width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))

height = max(height1, height2)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('abs.mp4', fourcc, 30.0, (width1 + width2 + 150, height))

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1 or not ret2:
        break

    frame1 = cv2.resize(frame1, (width1, height)) if ret1 else np.zeros((height, width1, 3), dtype='uint8')
    frame2 = cv2.resize(frame2, (width2, height)) if ret2 else np.zeros((height, width2, 3), dtype='uint8')

    label_space = np.zeros((height, 150, 3), dtype='uint8')
    cv2.putText(label_space, "<- with mediapipe", (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(label_space, "my approach ->", (10, height - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    combined_frame = np.concatenate((frame1, label_space, frame2), axis=1)
    out.write(combined_frame)

cap1.release()
cap2.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# import tkinter as tk
# from tkinter import filedialog

# def open_file_dialog():
#     root = tk.Tk()
#     root.withdraw()  # Hide the root window

#     file_path = filedialog.askopenfilename()  # Open a file dialog for manual file selection
#     if file_path:
#         print("Selected file:", file_path)
#         # Perform further operations with the selected file

# video = open_file_dialog()  # Call the function to open the file dialog